In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# !pip install --upgrade segmentation_models_pytorch

In [2]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [3]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
seed_everything(42)

In [5]:
folder = "train"  # "validation"
    
NTB = 4 

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [7]:
import torchvision.transforms as T


class ContrailDataset:
    def __init__(self, df, transform=None, normalize=False):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.normalize=normalize
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        

        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
            
#         return torch.tensor(image), torch.tensor(label)
    
        if self.normalize:
            image = self.normalize_image(torch.tensor(image))
            return image
        else:
            return torch.tensor(image), torch.tensor(label)
    
    
# class ContrailDataset:
#     def __init__(self, df, transform=None):
#         self.df = df  
#         self.images = df['image']
#         self.transform =transform
        
        
#     def __len__(self):
#         return len(self.images)
        
#     def __getitem__(self, idx):
#         image = np.load("../../input/" + self.images[idx]).astype(float)   
# #         label = np.load("../../input" + self.labels[idx]).astype(float)
        
        
#         # label_cls = 1 if label.sum() > 0 else 0
#         if self.transform :
#             data = self.transform(image=image)
#             image  = data['image']
#             image = np.transpose(image, (2, 0, 1))
            
#         return torch.tensor(image)

In [8]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model1(cfg):
    model = TimmSegModel(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model2(cfg):
    model = TimmSegModel2(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


##################

n_blocks = 4

class TimmSegModel3(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel3, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model3(cfg):
    model = TimmSegModel3(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model



class TimmSegModel4(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel4, self).__init__()

        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 512, 512))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
def load_model4(cfg):
    model = TimmSegModel4(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model







## Rohit Model 
class Net_R(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],     
            encoder_weights=None,   
            in_channels=3,  
            classes=cfg["num_classes"],
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask
    
    
def load_modelr1(cfg):
    model = Net_R(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model



class TimmSegModelR1(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModelR1, self).__init__()

        self.n_blocks = 4
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:self.n_blocks+1],
                decoder_channels=decoder_channels[:self.n_blocks],
                n_blocks=self.n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[self.n_blocks-1], 
                      cfg['num_classes'],
                      kernel_size=(3, 3),
                      stride=(1, 1),
                      padding=(1, 1)
            ),
            nn.UpsamplingBilinear2d(scale_factor=1)
        )

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:self.n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features


    
def load_modelr2(cfg):
    model = TimmSegModelR1(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model

In [9]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

def get_transform2(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_LINEAR),
    ], p=1.0)
    return transform




In [10]:
# val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
# val_df_full = val_df.copy()
# val_dups = np.load("../../input/data_utils/dups_val.npy")
# val_dups = [int(val_id) for val_id in val_dups]

# val_df = val_df.loc[~val_df['id'].isin(val_dups)].reset_index(drop=True)
# print(val_df.shape, val_df_full.shape)

In [11]:
# val_df = pd.read_csv(f'../../input/pseudo/{folder}_data_{NTB}.csv') 

val_df = pd.read_csv('../../input/data_utils/train_5_folds.csv')
val_df['label_many'] = val_df.image.apply(lambda x:  f"labels_many/train_data/{x.split('/')[-2]}/label_many.npy")


In [12]:
CFGS1 = [
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s3/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_07_tta",
        'model_func': load_modelr1,
        'tta': True,
        'normalize': False
    }
    
]

In [51]:
final_preds = []

for idx, cfg in enumerate(CFGS1):   

#     if idx <= 7:
#         continue
        
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    val_transform2 = get_transform2(cfg['img_size'])

    valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=False)  
    if cfg['normalize'] and ("ioa_" in cfg['call_sign']):
        valid_dataset = ContrailDataset(val_df, transform=val_transform2, normalize=True)  

    
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 4, 
        pin_memory = True, 
        drop_last = False
    )
    
    
    model_base = cfg['model_func'](cfg)        
      
    if cfg['tta']:
        if "roh_" in cfg['call_sign']:
            model = tta.SegmentationTTAWrapper(model_base, tta.aliases.flip_transform(), merge_mode='mean')
        else:
            model = tta.SegmentationTTAWrapper(model_base, tta.aliases.hflip_transform(), merge_mode='mean')
    
    
    model.to(device)
    model.eval()
    

    preds = []
    masks_ = []
        
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  

        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 
        masks_.append(torch.squeeze(masks, dim=1))
        with torch.inference_mode():
            images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
            pred = model(images).sigmoid()   
            pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest')
            preds.append(torch.squeeze(pred, dim=1))
        
            
            
    model_preds = torch.cat(preds, dim=0).detach().cpu()  
#     torch.save(model_preds, f"../../output/pseudo_preds{NTB}/{folder}/{cfg['call_sign']}.pt")    

    
    
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
        
#     model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
#     model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
#     # save
#     torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}.pt")    
    
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 

        
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold
    
        
#     print(best_dice_score, best_threshold)
#     final_preds1.append(model_preds)
    
    
    final_preds.append(model_preds)
    
    if cfg['tta']: del model
    del model_base
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s3/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.24, 'call_sign': 'roh_07_tta', 'model_func': <function load_modelr1 at 0x7fb7c2520dc0>, 'tta': True, 'normalize': False}


  0%|          | 0/642 [00:00<?, ?it/s]

RuntimeError: Parent directory ../../output/pseudo_preds4/train does not exist.

In [56]:
# best_threshold = 0.0
# best_dice_score = 0.0
# for threshold in [i / 100 for i in range(101)] :
#     score = dice_coef(model_masks.detach().cpu(), model_preds.detach().cpu(), thr=threshold).cpu().detach().numpy() 
#     if score > best_dice_score:
#         best_dice_score = score
#         best_threshold = threshold

print("Global Dice", best_dice_score, best_threshold)
score = dice_coef(model_masks.detach().cpu(), model_preds.detach().cpu(), thr=0.5).cpu().detach().numpy() 
print("Global Dice", score, 0.5)

scores = []
for pred, mask in  zip(model_preds, model_masks):
    scores.append(dice_coef(mask, pred, thr=0.5).cpu().detach().numpy())
        

Global Dice 0.71164286 0.04
Global Dice 0.7223227 0.5


In [58]:
val_df['scores'] = scores

In [60]:
val_df.to_csv("../../input/data_utils/train_5_folds.csv", index=False)

In [62]:
df = val_df.loc[val_df.scores < 0.5].reset_index(drop=True)

In [63]:
# df.scores.value_counts()
df['class'].value_counts()

1    1446
0     694
Name: class, dtype: int64

In [45]:
df

,image,label,class,label_many,id,fold,scores
0,train_data/5747024955383921098/image.npy,train_data/5747024955383921098/label.npy,1,labels_many/train_data/5747024955383921098/lab...,5747024955383921098,1.0,1.086956e-08
1,train_data/432548070956515051/image.npy,train_data/432548070956515051/label.npy,1,labels_many/train_data/432548070956515051/labe...,432548070956515051,0.0,5.151515e-01
2,train_data/3300848841428176436/image.npy,train_data/3300848841428176436/label.npy,1,labels_many/train_data/3300848841428176436/lab...,3300848841428176436,4.0,4.999998e-07
3,train_data/3374031604530790277/image.npy,train_data/3374031604530790277/label.npy,1,labels_many/train_data/3374031604530790277/lab...,3374031604530790277,0.0,5.959184e-01
4,train_data/9131223346423158425/image.npy,train_data/9131223346423158425/label.npy,1,labels_many/train_data/9131223346423158425/lab...,9131223346423158425,1.0,7.246377e-09
...,...,...,...,...,...,...,...
2915,train_data/489440619736314180/image.npy,train_data/489440619736314180/label.npy,1,labels_many/train_data/489440619736314180/labe...,489440619736314180,0.0,5.875706e-01
2916,train_data/1392568856138232946/image.npy,train_data/1392568856138232946/label.npy,1,labels_many/train_data/1392568856138232946/lab...,1392568856138232946,4.0,5.723370e-01
2917,train_data/4281530218709087779/image.npy,train_data/4281530218709087779/label.npy,1,labels_many/train_data/4281530218709087779/lab...,4281530218709087779,3.0,5.195531e-01
2918,train_data/5444185206942894271/image.npy,train_data/5444185206942894271/label.npy,1,labels_many/train_data/5444185206942894271/lab...,5444185206942894271,2.0,3.829788e-01


In [18]:
# NTB = 3
call_signs = [
    "roh_01_tta", "nir_01_tta", "nir_02_tta", "nir_03_tta", "nir_04_tta", "ioa_01", "ioa_02"
]


final_preds = []
for sign in tqdm(call_signs, total=len(call_signs)):
    wt = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/pseudo_preds{NTB}/{folder}/{sign}.pt") 
    final_preds.append(wt)


  0%|          | 0/7 [00:00<?, ?it/s]

In [19]:
final_preds = torch.stack(final_preds).mean(dim=0)
final_preds = (final_preds>0.35).double()

In [20]:
val_df['id'] = val_df['image'].apply(lambda x: x.split("/")[-2])

In [21]:
ids = val_df['id'].values
for (val, label_id) in tqdm(zip(final_preds, ids), total=len(ids)): 
    mask = val.view(256, 256, 1).detach().cpu().numpy()
    np.save(f"../../input/pseudo/{folder}_data_{NTB}/{label_id}/label_6811_702lb.npy", mask.astype('float16')) 


  0%|          | 0/1856 [00:00<?, ?it/s]

In [21]:
# 0.657336 0.01


# call_signs1 = [
#     "roh_01_tta", "nir_01_tta", "nir_02_tta", "nir_03_tta", "nir_04_tta"
# ] 

# call_signs1 = [
#     "roh_02_tta",  "roh_03_tta", "roh_04_tta", "roh_05_tta", "roh_06_tta",  "roh_07_tta",  "roh_08_tta"  
# ] 

call_signs1 = [
    "roh_01_tta", 
    "nir_01_tta",
    "nir_02_tta",
    "nir_03_tta",
    "nir_04_tta",
    
    "ioa_01", 
    "ioa_02",
    
] 

model_masks = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.pt")


preds1 = []

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    wt = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.pt")    
    preds1.append(wt)
    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    
    
# preds1.append(preds2)
    
final_preds = preds1
final_preds = torch.stack(final_preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

print("0.5 TH Score: ", score)


best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
print(best_dice_score, best_threshold)

  0%|          | 0/7 [00:00<?, ?it/s]

0.6510141
0.66153294
0.649828
0.6697093
0.6561677
0.66287154
0.6537375
0.5 TH Score:  0.6811164
0.6867434 0.35


In [19]:
# 0.5 TH Score:  0.67887855
# 0.6856482 0.28



# 0.5 TH Score:  0.6811164
# 0.6867434 0.35
